## Toy Example

In [1]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.cluster import AgglomerativeClustering
import plotly.express as px
from sklearn.cluster import DBSCAN

import os

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import numpy as np # linear algebra
from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from keras.layers import Input, Dense, merge
from keras.models import Model
from tensorflow.python.client import device_lib
import keras
from keras import optimizers
from keras.layers.advanced_activations import LeakyReLU



Using TensorFlow backend.


# Para adicionar

Possiveis adicionais:

- Soro
- sodio
- Exercises
- Genetics consult
- VASOPRESSIN
- PULSE
- caffeine bolus
- position
- Resp. Rate
- Stress



More: 

- viagra
- disturbio hormonal
- doença renal
- tabagismo
- alcoolismo
- tendencia hereditária?

In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
data = pd.read_csv('./tables/gtc_aw_dian_str.csv')

data = data[np.logical_or(data['"ARTERIAL_SIS"']<70,data['"ARTERIAL_DIAS"']<45)] #= data.sample(300000) #= data.iloc[:100000]#.drop(['"CHARTTIME"_x_x','"CHARTTIME"_y.1','"CHARTTIME"','"CHARTTIME"_x_y'],axis = 1, inplace = True)

data.drop(
    ['"SUBJECT_ID"_x', '"SUBJECT_ID"_y', '"HADM_ID"', '"ICD9_CODE"','"ICUSTAY_ID"', 
     'index', '"CHARTTIME"_x','"CHARTTIME"_y', '"DAYTIME"'],
    axis = 1, inplace = True)

In [4]:
data = data[data['"GLUCOSE"']<1000]

In [5]:
data.drop_duplicates(inplace=True)


In [6]:
data = data.fillna(0)

In [14]:
data.columns

Index(['Unnamed: 0_x', '"ROW_ID"_x', '"SUBJECT_ID"_x', '"ICUSTAY_ID"',
       '"CHARTTIME"_x', '"SUBJECT_ID"_y', '"HADM_ID"_x', '"CHOLES"',
       '"SUBJECT_ID"_x.1', '"HADM_ID"_y', '"TRIGLY"', '"DATEDAY"_x',
       '"SUBJECT_ID"_y.1', '"HADM_ID"', '"GENDER"', '"ADMISSION_AGE"',
       '"ETHNICITY"', '"HEIGHT"', 'Unnamed: 0_y', '"ROW_ID"_y',
       '"CHARTTIME"_y', '"WEIGHT"', '"DATEDAY"_y'],
      dtype='object')

#Y1 = np.round(data['"ARTERIAL_SIS"']/10)
#Y2 = np.round(data['"ARTERIAL_DIAS"']/10)
#data['mean'] = np.round(data['mean']/10)
Y1 =data['"ARTERIAL_SIS"']
Y2 = data['"ARTERIAL_DIAS"']
data.drop(['"ARTERIAL_DIAS"', '"ARTERIAL_SIS"'] ,axis = 1, inplace = True)
X = data
X2 = data

In [8]:
Y1 = data['"GLUCOSE"']
#Y2 = np.round(data['"ARTERIAL_DIAS"']/10)
#data['mean'] = np.round(data['mean']/10)

#Y2 = data['"ARTERIAL_DIAS"']
data.drop(['"GLUCOSE"'] ,axis = 1, inplace = True)
X = data
X2 = data

Dividimos os dados em treino e teste.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
     X, Y1, test_size=0.33, random_state=42)

X2_train, X2_test, y2_train, y2_test = train_test_split(
     X2, Y2, test_size=0.33, random_state=42)

ys_train = pd.DataFrame(np.transpose([y_train, y2_train]), columns = ['y1','y2'])
ys_test =  pd.DataFrame(np.transpose([y_test, y2_test]), columns = ['y1','y2'])

Treinamos um modelo de rede neural

In [10]:
model = MLPRegressor(hidden_layer_sizes=(128,128,128,128,128,128,128,128),activation='relu',max_iter=1500, verbose=1, solver='adam',  n_iter_no_change=50)

In [11]:
model.fit(X_train,y_train)

ValueError: could not convert string to float: '2114-01-28 01:00:00'

Obtendo o R² de treino:

In [12]:
model.score(X_train,y_train)

0.7445626409621711

E o R² de teste:

In [13]:
model.score(X_test,y_test)

0.5223824386660867

In [14]:
y_pred = model.predict(X_test)

y_train_pred = model.predict(X_train)

In [15]:
import matplotlib.pyplot as plt
plt.scatter(y_train_pred,y_train,  c="g", alpha=0.5, label="Luck")

In [16]:
y_pred[:10]

array([227.06025223, 109.18217038, 108.72842318, 253.61257183,
       124.34366621, 188.86311766, 192.76225149, 104.33720251,
        68.46947981, 125.08886661])

In [17]:
y_test[:10]

7513    226.0
7488    124.0
2472     90.0
4517    248.0
2983    125.0
6087    372.0
6463    103.0
6365    119.0
6205    196.0
4258    130.0
Name: "GLUCOSE", dtype: float64

Calculamos o erro de predição.

In [18]:
r2_score(y_pred, y_test)

0.32774578357790163

In [19]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

print(mean_absolute_error(y_train,y_train_pred)/y_train.mean())
print(mean_squared_error(y_train,y_train_pred)/y_train.mean())
mean_absolute_error(y_train,y_train_pred), mean_absolute_error(y_test,y_pred)

0.1329813641432261
6.714409444592111


(19.569480406887244, 27.486300002698297)

In [20]:
model_error =   y_pred -y_test
train_error = y_train - y_train_pred

In [21]:
X_train['error'] = train_error
X_test['error'] = model_error
X_train['y'] = y_train
X_test['y'] = y_test

/home/guilherme/anaconda3/envs/tf36/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/guilherme/anaconda3/envs/tf36/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/guilherme/anaconda3/envs/tf36/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [22]:
model2 = MLPRegressor(hidden_layer_sizes=(128,128,128,128,128,128,128,128),activation='relu',max_iter=2000,verbose=1,n_iter_no_change = 50, validation_fraction= 0.2, early_stopping = True)

In [23]:
model2.fit(X2_train,y2_train)

NameError: name 'X2_train' is not defined

Obtendo o R² de treino:

In [ ]:
model2.score(X2_train,y2_train)

In [ ]:
model2.score(X2_test, y2_test)

In [ ]:
y2_test.iloc[:10]

In [ ]:
X2_test.iloc[:10]

In [ ]:
X2_train.iloc[0]

In [ ]:
y2_train.iloc[0]

E o R² de teste:

In [ ]:
X2_test

In [ ]:
model2.score(X2_test,y2_test)

curve = pd.DataFrame(model2.loss_curve_)
curve.plot(figsize = (15,10))

In [ ]:
y2_pred = model2.predict(X2_test)
y2_train_pred = model2.predict(X2_train)

In [ ]:
np.mean(abs(y2_test - y2_pred))

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mean_absolute_error(y2_train,y2_train_pred), mean_absolute_error(y2_test,y2_pred)
print(mean_absolute_error(y2_train,y2_train_pred)/y2_train.mean())
print(mean_squared_error(y2_train,y2_train_pred)/y2_train.mean())

In [ ]:

print(mean_absolute_error(y2_train,y2_train_pred)/y2_train.mean())
print(mean_squared_error(y2_train,y2_train_pred)/y2_train.mean())

In [ ]:
print(mean_absolute_error(y2_test,y2_pred)/y2_test.mean())
print(mean_squared_error(y2_test, y2_pred)/y2_test.mean())

In [ ]:
2.7956278576615126/y2_test.mean()

In [ ]:
y2_pred.mean()

In [ ]:
y2_test.mean()

Calculamos o erro de predição.

In [ ]:
model2_error =   y2_pred -y2_test
train2_error = y2_train - y2_train_pred

In [ ]:
X2_train['error'] = train2_error
X2_test['error'] = model2_error
X2_train['y'] = y2_train
X2_test['y'] = y2_test

In [ ]:
X_train['error2'] = train2_error
X_test['error2'] = model2_error
X_train['y2'] = y2_train
X_test['y2'] = y2_test

In [ ]:
X_train.drop(['30306.0', '30307.0', '30309.0'], axis = 1, inplace = True)
X_test.drop(['30306.0', '30307.0', '30309.0'], axis = 1, inplace = True)

In [ ]:
X_train.to_csv('X_train_short.csv', )
X_test.to_csv('X_test_short.csv')

X_train = pd.read_csv('X_train.csv')
X_train.drop(['Unnamed: 0'], axis = 1, inplace = True)
X_test = pd.read_csv('X_test.csv')
X_test.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
df_encoder_train = X_train.copy()#.merge(X2_train,left_index=True,right_index=True)
df_encoder_test = X_test.copy()#.merge(X2_test,left_index=True,right_index=True)

In [ ]:
df_encoder_test['221749.0'].unique()#.columns

In [ ]:
#verificando se sem os erros fica melhor
df_outs_train = df_encoder_train[['y','y2']]
df_outs_test = df_encoder_test[['y','y2']]
df_features_train = df_encoder_train.drop(['error','error2','y','y2'], axis = 1)
df_features_test = df_encoder_test.drop(['error','error2','y','y2'], axis = 1)


df_outs_train['y'] = df_outs_train['y']/df_outs_train['y'].max()
df_outs_train['y2'] = df_outs_train['y2']/df_outs_train['y2'].max()

In [ ]:
#df_outs_train = df_outs_train[['y','y2']]
#df_encoder_train = df_encoder_train[['"HEART_RATE"','"PPT"','error',	'y',	'error2',	'y2']]

In [ ]:
encoding_dim = 1
ncol = 26
keras.backend.clear_session()


In [ ]:
keras.backend.clear_session()
input_dim = Input(shape = (ncol, ))
input_dim2 = Input(shape = (ncol - 4, ))
# Encoder Layers
encoded1 = Dense(30, activation = 'relu',kernel_initializer='random_normal')(input_dim)
encoded2 = Dense(20, activation = 'relu',kernel_initializer='random_normal')(encoded1)
encoded3 = Dense(10, activation = 'relu',kernel_initializer='random_normal')(encoded2)
encoded4 = Dense(5, activation = 'relu',kernel_initializer='random_normal')(encoded3)
encoded5 = Dense(1, activation = 'relu',kernel_initializer='random_normal')(encoded4)
encoded13 = Dense(encoding_dim, activation = 'relu',kernel_initializer='random_normal')(encoded5)
merged = keras.layers.concatenate([encoded13, input_dim2], axis=-1)
# Decoder Layers
decoded1 = Dense(100, activation = 'relu',kernel_initializer='random_normal')(merged)
decoded2 = Dense(500, activation = 'relu',kernel_initializer='random_normal')(decoded1)
decoded3 = Dense(1000, activation = 'relu',kernel_initializer='random_normal')(decoded2)
decoded4 = Dense(2500, activation = 'relu',kernel_initializer='random_normal')(decoded3)
decoded5 = Dense(5000, activation = 'relu',kernel_initializer='random_normal')(decoded4)
decoded13 = Dense(2, activation = 'linear',kernel_initializer='random_normal')(decoded5)

# Combine Encoder and Deocder layers
autoencoder = Model(inputs = [input_dim, input_dim2], outputs = [decoded13])
sgd = optimizers.SGD(lr=0.01)

# Compile the Model
autoencoder.compile(optimizer = 'adam', loss = 'mean_absolute_error')

In [ ]:
df_encoder_train.columns

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=15,
                              verbose=0, mode='auto')


callbacks = [es]
autoencoder.fit(x = [df_encoder_train, df_features_train], y = df_outs_train, epochs = 30, 
                batch_size = 32, shuffle = True,
               callbacks = callbacks
                
               )

In [ ]:
encoder = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
encoded_input = Input(shape = (encoding_dim, ))


In [ ]:
df_features_train.nunique()

In [ ]:
feata = '"HEART_RATE"'

In [ ]:
idn = 7741
df_features_train.iloc[idn]

In [ ]:
df_outs_train.iloc[idn]

In [ ]:
df_features_train[feata] = df_features_train[feata] -10

In [ ]:
df_features_train[feata] = df_features_train[feata] +10
#df_encoder_train['y'].iloc[10:20]#['X1_x']# = df_features_train.iloc[0:2]['X1_x']

In [ ]:
encoded_train = pd.DataFrame(encoder.predict([df_encoder_train,df_features_train]))
encoded_train = encoded_train.add_prefix('feature_')

In [ ]:
encoded_train['feature_1'].iloc[idn]

In [ ]:
encoded_train['feature_1'].iloc[idn]

In [ ]:
r2_score(df_outs_train, encoded_train)

In [ ]:
df_outs_train

ys_train.iloc[0:2]

In [ ]:
X_t = df_features_train

In [ ]:
contra = X_t['X0_x']*X_t['X1_x']+X_t['X2_x']*X_t['X3_x']+X_t['X1_x']*X_t['X3_x'] + x4_train*((X_t['X1_x'])-X_t['X2_x'])

In [ ]:
contra.head()

In [ ]:
r2_score(contra, encoded_train['feature_3'])

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [ ]:
mse = mean_absolute_error(contra, encoded_train['feature_3'])
mse

In [ ]:
mpe_sum = 0
for real, prediction in zip(contra, encoded_train['feature_3']):
    mpe_sum += ((real - prediction)/real)
mpe = mpe_sum/len(contra)
print(mpe)

In [ ]:
mse / np.mean(contra)

In [ ]:
mse / np.std(contra)

In [ ]:
np.mean(contra)

In [ ]:
np.std(contra)